<a href="https://colab.research.google.com/github/mashyko/NLP_Mecab/blob/master/Mecab_Neologd_livedoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MecabとGensim による類似単語検出：Livedoor News を使用

Mecabとmecab-ipadic-neologdのインストール： 

NEologd辞書の追加　:
NEologdは、Web上から得た新語に対応しており、毎週更新されるMeCab用のシステム辞書です。

mecab-python もインストール


In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


mecab-ipadic-neologdが保存されているフォルダーを調べる。

In [2]:
!mecab-config --dicdir

/usr/lib/x86_64-linux-gnu/mecab/dic


In [3]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [4]:
import MeCab
sent = "スマホでインスタが流行っている" 
tagger = MeCab.Tagger ()
print(tagger.parse(sent))

スマホ	名詞,一般,*,*,*,*,*
で	助詞,格助詞,一般,*,*,*,で,デ,デ
インスタ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
流行っ	動詞,自立,*,*,五段・ラ行,連用タ接続,流行る,ハヤッ,ハヤッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS



In [5]:
!echo "SMAPのキムタクはポケモンに会いに行き、スマホを売るバイトをしている" | mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

SMAP	名詞,固有名詞,一般,*,*,*,SMAP,スマップ,スマップ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
キムタク	名詞,固有名詞,一般,*,*,*,キムタク,キムタク,キムタク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ポケモン	名詞,固有名詞,一般,*,*,*,Pokemon!,ポケモン,ポケモン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
会い	名詞,一般,*,*,*,*,会意,カイイ,カイー
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
行き	動詞,自立,*,*,五段・カ行促音便,連用形,行く,イキ,イキ
、	記号,読点,*,*,*,*,、,、,、
スマホ	名詞,固有名詞,一般,*,*,*,スマホ,スマホ,スマホ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
売る	動詞,自立,*,*,五段・ラ行,基本形,売る,ウル,ウル
バイト	名詞,一般,*,*,*,*,バイト,バイト,バイト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS


ここからLivedoor News を使用しますので、以下のようにファイルをダウンロードして配置してください。

/My Drive/NLP/livedoor-news-data/topic-news.xml 

この文書が入ったMy Drive にmountします。

In [6]:
! git clone https://github.com/mashyko/NLP_Mecab
%cd NLP_Mecab
!ls 

Cloning into 'NLP_Mecab'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 137 (delta 13), reused 0 (delta 0), pack-reused 102
Receiving objects: 100% (137/137), 12.59 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/NLP_Mecab
ChatBot_1.ipynb       gensim_topic.ipynb	  Mecab_Intro.ipynb
corpora		      gensim_tutor_English.ipynb  MeCab.ipynb
datasets	      livedoor-news-data	  Mecab_janome.ipynb
fasttext_intro.ipynb  Livedoor-text		  Mecab_Neologd.ipynb
Gensim_doc2vec.ipynb  MeCab_1.ipynb		  README.md


分かち書きファイルの作成と保存

In [0]:
!mecab  -Owakati ./livedoor-news-data/topic-news.txt -o ./livedoor-news-data/topic-wakati.txt

gemsim向けのトークン化されたファイルの作成

In [0]:
import MeCab
tagger = MeCab.Tagger('-F\s%f[6] -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

raw = open('./livedoor-news-data/topic-news.xml', 'r')
text = open('./livedoor-news-data/topic-news_text.xml', 'w')

line = raw.readline()
while line:
    result = tagger.parse(line)
    text.write(result[1:]) # skip first \s
    line = raw.readline()

raw.close()
text.close()

In [9]:
!ls livedoor-news-data/

topic-news_text.xml  topic-news.txt  topic-news.xml


Gensimのインストール

In [10]:
!pip install gensim

word2vec による辞書の作成

In [11]:
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = word2vec.LineSentence('./livedoor-news-data/topic-news_text.xml')
model = word2vec.Word2Vec(sentences,
                          sg=1,
                          size=100,
                          min_count=1,
                          window=10,
                          hs=1,
                          negative=0)
model.save('./livedoor-news-data/topic-news.model')


2020-06-15 01:54:51,122 : INFO : collecting all words and their counts
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-06-15 01:54:51,125 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-15 01:54:51,209 : INFO : PROGRESS: at sentence #10000, processed 215781 words, keeping 14028 word types
2020-06-15 01:54:51,275 : INFO : collected 19365 word types from a corpus of 377586 raw words and 16734 sentences
2020-06-15 01:54:51,276 : INFO : Loading a fresh vocabulary
2020-06-15 01:54:51,313 : INFO : effective_min_count=1 retains 19365 unique words (100% of original 19365, drops 0)
2020-06-15 01:54:51,314 : INFO : effective_min_count=1 leaves 377586 word 

類似の単語を20個検出します。ここでは、「嵐」

In [12]:
model = word2vec.Word2Vec.load('./livedoor-news-data/topic-news.model')
model.wv.most_similar(positive=['嵐'],topn=20)

2020-06-15 01:55:05,087 : INFO : loading Word2Vec object from ./livedoor-news-data/topic-news.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-06-15 01:55:05,335 : INFO : loading wv recursively from ./livedoor-news-data/topic-news.model.wv.* with mmap=None
2020-06-15 01:55:05,336 : INFO : setting ignored attribute vectors_norm to None
2020-06-15 01:55:05,337 : INFO : loading vocabulary recursively from ./livedoor-news-data/topic-news.model.vocabulary.* with mmap=None
2020-06-15 01:55:05,338 : INFO : loading trainables recursively from ./livedoor-news-data/topic-news.model.trainables.* with mmap=None
2020-06-15 01:55:05,338 : INFO : setting ignored attribute cum_table to None


[('生歌', 0.8238643407821655),
 ('機材', 0.7589198350906372),
 ('続々', 0.6356038451194763),
 ('FNS歌謡祭', 0.6247184872627258),
 ('吹き荒れる', 0.6204281449317932),
 ('騒然', 0.6174483299255371),
 ('板野友美', 0.6095362901687622),
 ('酷', 0.6053171157836914),
 ('批判デモ', 0.5994373559951782),
 ('轟々', 0.5975573062896729),
 ('婦人公論', 0.5893675088882446),
 ('吉高由里子', 0.5866955518722534),
 ('偏向報道', 0.5862089395523071),
 ('めちゃ×2イケてる', 0.5842294692993164),
 ('小嶋陽菜', 0.5830409526824951),
 ('めろ', 0.5812135338783264),
 ('振舞い', 0.577261209487915),
 ('またもや', 0.5765401721000671),
 ('韓流ゴリ押し', 0.574148416519165),
 ('口移し', 0.5711820125579834)]

以上